In [9]:
# Dependencies
import numpy as np
import pandas as pd
import requests
import json

from config import census_key

In [10]:
#API Doc: https://www.census.gov/data/developers/data-sets/acs-5year.html
#Census Variable: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b

In [11]:
# Due to Census' API restraints this code is only valid for 2009-2018. 
# Below list must remain in chronological order
years = [2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]

# Params to be passed into API call, *DO NOT INCLUCE SPACES
census_variables = "B01003_001E" 

'''
CENSUS VARIABLE DESCRIPTIONS:
B01003_001E,	Total population,
B02001_002E,	Population (White Alone),
B02001_003E,	Population (Black or African American Alone),
B02001_004E,	Population (American Indian or Alaskan Native Alone),
B02001_005E,	Population (Asian Alone),
B02001_006E,	Population (Native Hawaiian and Other Pacific Islander Alone),
B02001_008E,	Population (Two or more races),
B03001_003E,	Population (Hispanic Origin)
'''

'\nCENSUS VARIABLE DESCRIPTIONS:\nB01003_001E,\tTotal population,\nB02001_002E,\tPopulation (White Alone),\nB02001_003E,\tPopulation (Black or African American Alone),\nB02001_004E,\tPopulation (American Indian or Alaskan Native Alone),\nB02001_005E,\tPopulation (Asian Alone),\nB02001_006E,\tPopulation (Native Hawaiian and Other Pacific Islander Alone),\nB02001_008E,\tPopulation (Two or more races),\nB03001_003E,\tPopulation (Hispanic Origin)\n'

In [12]:
#This blank list will hold the yearly API results
results = []

for year in years: 
    
    #API base url is dependent on year
    if year == 2009: #2009 Detailed Tables API Call
        url = f'https://api.census.gov/data/{year}/acs5?key={census_key}&get={census_variables},NAME&for=state:*'      

    elif year == 2010: #2010 Detailed Tables API Call     
        url = f'https://api.census.gov/data/{year}/acs/acs5?key={census_key}&get={census_variables},NAME&for=state:*'
        
    else: #2011-2018 Detailed Tables API Call
        url = f'https://api.census.gov/data/{year}/acs/acs5?get=NAME,{census_variables}&for=state:*&key={census_key}'
        
        
    #Incorporating try/except to catch any year w/o data        
    try:
        print(f'Retrieving {year} data...')
        print(url)
        response = requests.get(url).json()
        results.append(response)
        print(f'{year} data successfully retrieved!')
        print('-------------------')
    
    except:
        print(f'Unable to retrieve {year} data...skipping')
        print('-------------------')
        
print('Data retrieval complete')

Retrieving 2009 data...
https://api.census.gov/data/2009/acs5?key=56fd45ad99936050c5b62b7ea0591503e40b8364&get=B03001_003E,NAME&for=state:*
2009 data successfully retrieved!
-------------------
Retrieving 2010 data...
https://api.census.gov/data/2010/acs/acs5?key=56fd45ad99936050c5b62b7ea0591503e40b8364&get=B03001_003E,NAME&for=state:*
2010 data successfully retrieved!
-------------------
Retrieving 2011 data...
https://api.census.gov/data/2011/acs/acs5?get=NAME,B03001_003E&for=state:*&key=56fd45ad99936050c5b62b7ea0591503e40b8364
2011 data successfully retrieved!
-------------------
Retrieving 2012 data...
https://api.census.gov/data/2012/acs/acs5?get=NAME,B03001_003E&for=state:*&key=56fd45ad99936050c5b62b7ea0591503e40b8364
2012 data successfully retrieved!
-------------------
Retrieving 2013 data...
https://api.census.gov/data/2013/acs/acs5?get=NAME,B03001_003E&for=state:*&key=56fd45ad99936050c5b62b7ea0591503e40b8364
2013 data successfully retrieved!
-------------------
Retrieving 201

In [16]:
#Converting api results into yearly dataframes and consolidating into one large df
for year in years:
    year_index = years.index(year)
    data = results[year_index][1:]
    headers = results[year_index][0]
    
    if year_index == 0:
        first_year = pd.DataFrame(data, columns = headers)
        first_year['Year'] = year
        first_year = first_year[sorted(first_year.columns)]
        census_df = first_year
        
    else:
        headers = results[year_index][0] #setting headers again bc column order is in consistent across year (ie '09 vs '16)
        next_year = pd.DataFrame(data, columns = headers)
        next_year['Year'] = year
        next_year = next_year[sorted(next_year.columns)]
        census_df = pd.concat([census_df, next_year], ignore_index=True)
        
census_df.head()

,B03001_003E,NAME,Year,state
0,39661,Alaska,2009,02
1,130220,Alabama,2009,01
2,153630,Arkansas,2009,05
3,1881878,Arizona,2009,04
4,13102161,California,2009,06


In [17]:
#convert api results to interger
census_df = census_df.astype({"B03001_003E" : 'int32'})

In [18]:
#rename variables
census_df = census_df.rename(columns={"B01003_001E" : "Total Population",
                                      'NAME':'State'})

#reorganize columns, dropping state code
census_df = census_df[['State','Year',"Total Population",]]

KeyError: "['Total Population'] not in index"

In [ ]:
'''#Calculate % of population
census_df["White (%)"] = round(census_df["White (pop)"]/census_df['Total Population'],3)*100
census_df["Black (%)"] = round(census_df["Black (pop)"]/census_df['Total Population'],3)*100
census_df["Indigenous (%)"] = round(census_df["Indigenous (pop)"]/census_df['Total Population'],3)*100
census_df["Asian (%)"] = round(census_df["Asian (pop)"]/census_df['Total Population'],3)*100
census_df["Pacfic Islander (%)"] = round(census_df["Pacfic Islander (pop)"]/census_df['Total Population'],3)*100
census_df["Mixed Race (%)"] = round(census_df["Mixed Race (pop)"]/census_df['Total Population'],3)*100
census_df["Hispanic (%)"] = round(census_df["Hispanic (pop)"]/census_df['Total Population'],3)*100'''

In [ ]:
census_df

In [ ]:
census_df.to_csv('../Resources/race_data.csv', index=False)